In [2]:
from PIL import Image
import numpy as np
import math

In [3]:
def load_image(filename: str, resize: tuple):
  img = Image.open(filename)
  img = img.resize(resize)
  return np.array(img)

In [4]:
def euclidean_distance(p1, p2):
    return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p1, p2)))

In [5]:
def manhattan_distance(p1, p2):
  return sum(abs(px - qx) for px, qx in zip(p1, p2))

In [6]:
print(euclidean_distance([3, 3], [6, 12]))

9.486832980505138


In [7]:
print(manhattan_distance((1, 0, 2, 3), (4, 4, 3, 1)))

5
